### Build a Simple LLM Application with LCEL

In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe


In [32]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Import and configure OpenAI
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

# Get Groq API key for open source models
groq_api_key = os.getenv("GROQ_API_KEY")

In [33]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x15432c450>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1540ecad0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [34]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to Hindi"),
    HumanMessage(content="Hello How are you?"),
]

result = model.invoke(messages)

In [35]:
result

AIMessage(content='नमस्ते, आप कैसे हैं? (Namaste, aap kaise hain?) \n\n\nThis translates directly to "Hello, how are you?". \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 21, 'total_tokens': 56, 'completion_time': 0.063636364, 'prompt_time': 0.002124797, 'queue_time': 0.017340229999999998, 'total_time': 0.065761161}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-750a8c2f-7d8c-4619-85ad-8689d4533467-0', usage_metadata={'input_tokens': 21, 'output_tokens': 35, 'total_tokens': 56})

In [36]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(result)

'नमस्ते, आप कैसे हैं? (Namaste, aap kaise hain?) \n\n\nThis translates directly to "Hello, how are you?". \n'

In [37]:
### Using LCEL- chain the components

chain = model | parser
chain.invoke(messages)

'नमस्ते, आप कैसे हैं? (Namaste, aap kaise hain?) \n\n\nThis is a polite and common way to say "Hello, how are you?" in Hindi. \n'

In [43]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Trnaslate the following into {language}:"

prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template), ("user", "{text}")]
)

In [47]:
##Chaining together components with LCEL
chain = prompt | model | parser
chain.invoke({"language": "Hindi", "text": "Hi"})

'नमस्ते (Namaste) \n'